In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

###### spark 

In [2]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

###### load clickstream 

In [3]:
clickstream = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_CLICKSTREAM, hive)

In [4]:
clickstream.count()

107172232

In [5]:
clickstream.show(3)

,inn,timestamp,hitPagePath,hitPageTitle,eventCategory,eventAction,eventLabel,product
0,470707811305,1623239609,/accounts-payments,СберБизнес,[std]: payments,click,[ruble_payment_edit]: [action.save],
1,6220008542,1623318711,/main,СберБизнес,[std]: main,click,"[layout.headergeneralsbbol3]: [open burger, novelty: true]",
2,5260170451,1623412583,/accounts-payments/account/:accountId,СберБизнес,[std]: payments,click,[ruble_payment_edit]: [action.save],


###### create item dict 

Теперь попробуем выделить в качестве целевых событий конкретный продукт

###### numeric orders actions 

In [6]:
w_event = Window().orderBy("eventAction", "eventCategory", "hitPagePath")

In [7]:
visit_event_id = clickstream.withColumn("event_id", f.rank().over(w_event))

In [8]:
%%time
max_event_id = visit_event_id.select(f.max("event_id")).take(1)[0]["max(event_id)"]

CPU times: user 27.6 ms, sys: 52.8 ms, total: 80.4 ms
Wall time: 3min 43s


In [9]:
w_group = Window().orderBy("product")

In [10]:
max_event_id

107172215

In [11]:
visit_group_event_id = visit_event_id.withColumn("prod_id", 
                                                 f.when(~f.col("product").like(""), 
                                                        max_event_id + f.rank().over(w_group)) \
                                                          .otherwise(None))

In [12]:
visit_item_id = visit_group_event_id.withColumn("item_id", f.coalesce("prod_id", "event_id"))

In [13]:
%%time
visit_item_id.show(3)

CPU times: user 88.7 ms, sys: 116 ms, total: 204 ms
Wall time: 9min 31s


,inn,timestamp,hitPagePath,hitPageTitle,eventCategory,eventAction,eventLabel,product,event_id,prod_id,item_id
0,3703016641,1623125750,/accounts-payments,СберБизнес - версия: 41.000.04_0005,None,click,/accounts-payments,,1,None,1
1,6379002135,1622454978,/main,СберБизнес,[std]: tasks: payroll,click,[mass sign item - 4]: [action.send],,106569081,None,106569081
2,7716834257,1623156404,/accounts-payments,СберБизнес - версия: 41.000.04_0005,None,click,/accounts-payments,,1,None,1


###### first time action 

In [14]:
visit_item_id = visit_item_id.select("inn",
                                     "timestamp",
                                     "hitPagePath",
                                     "eventCategory",
                                     "eventAction",
                                     "product",
                                     "event_id",
                                     "prod_id",
                                     "product",
                                     "item_id").distinct()

In [15]:
cols = [f.col(col) for col in visit_item_id.columns if col != "timestamp"]

In [16]:
visit_unique = visit_item_id.groupBy(*cols).agg(f.min("timestamp"))

In [17]:
visit_unique.cache()

DataFrame[inn: string, hitPagePath: string, eventCategory: string, eventAction: string, product: string, event_id: int, prod_id: int, product: string, item_id: int, min(timestamp): bigint]

###### create item stats 

In [18]:
%%time
visit_unique.show(3)

CPU times: user 204 ms, sys: 252 ms, total: 457 ms
Wall time: 20min 44s


,inn,hitPagePath,eventCategory,eventAction,product,event_id,prod_id,product,item_id,min(timestamp)
0,910708469746,/shop/activated-products,[std]: shop,click,accounts_and_payments,105012110,214195630,accounts_and_payments,214195630,1622409050
1,773574877403,/shop/activated-products,[std]: shop,click,accounts_and_payments,105012110,214195630,accounts_and_payments,214195630,1622411845
2,781000050331,/shop/activated-products,[std]: shop,click,accounts_and_payments,105012110,214195630,accounts_and_payments,214195630,1622413660


In [19]:
%%time
visit_unique.select("item_id").distinct().count()

CPU times: user 2.9 ms, sys: 6.05 ms, total: 8.95 ms
Wall time: 1.66 s


15841

In [20]:
num_of_users = visit_unique.select("inn").distinct().count()

In [21]:
w_event_stats = Window().partitionBy("event_id")

In [22]:
visit_event_stats = visit_unique.withColumn("event_user_count", f.count("inn").over(w_event_stats)) \
                                .withColumn("event_user_percent", 
                                            f.format_number(f.col("event_user_count") / num_of_users * 100, 5) \
                                             .cast(stypes.DoubleType()))

In [23]:
w_group_stats = Window().partitionBy("prod_id")

In [24]:
visit_group_stats = visit_event_stats.withColumn("prod_user_count", 
                                                 f.when(f.col("prod_id").isNotNull(), 
                                                        f.count("inn").over(w_group_stats))
                                                  .otherwise(None))\
                                      .withColumn("prod_user_percent", 
                                                  f.when(f.col("prod_id").isNotNull(), 
                                                         f.format_number(f.col("prod_user_count") / num_of_users * 100, 5) \
                                                          .cast(stypes.DoubleType())) \
                                                   .otherwise(None))

In [25]:
w_item_stats = Window().partitionBy("item_id")

In [26]:
visit_item_stats = visit_group_stats.withColumn("item_user_count", f.count("inn").over(w_item_stats)) \
                                    .withColumn("item_user_percent", 
                                                f.format_number(f.col("item_user_count") / num_of_users * 100, 5) \
                                                 .cast(stypes.DoubleType()))

In [27]:
visit_item_stats.show(3)

,inn,hitPagePath,eventCategory,eventAction,product,event_id,prod_id,product,item_id,min(timestamp),event_user_count,event_user_percent,prod_user_count,prod_user_percent,item_user_count,item_user_percent
0,5257197100,/accounts-payments,None,click,,1,None,,1,1622408895,3710,0.35721,None,None,3710,0.35721
1,6407003278,/accounts-payments,None,click,,1,None,,1,1622410033,3710,0.35721,None,None,3710,0.35721
2,1657033678,/accounts-payments,None,click,,1,None,,1,1622412259,3710,0.35721,None,None,3710,0.35721


In [28]:
item_dict = item_stats = visit_item_stats.select("hitPagePath", "eventCategory", "eventAction", "product",
                               "event_id", "event_user_count", "event_user_percent",
                               "prod_id", "prod_user_count", "prod_user_percent",
                               "item_id", "item_user_count", "item_user_percent") \
                       .distinct()

In [29]:
item_dict.show(5)

,hitPagePath,eventCategory,eventAction,product,event_id,event_user_count,event_user_percent,prod_id,prod_user_count,prod_user_percent,item_id,item_user_count,item_user_percent
0,/accounts-payments,None,click,,1,3710,0.35721,None,None,None,1,3710,0.35721
1,/accounts-payments/RUB_COMM/:id,None,click,,9928,1,0.00010,None,None,None,9928,1,0.00010
2,/accounts-payments/account/:accountId,None,click,,9929,5323,0.51251,None,None,None,9929,5323,0.51251
3,/accounts-payments/orders/sign,None,click,,21640,273,0.02629,None,None,None,21640,273,0.02629
4,/accounts/kartoteka/:id/:cardType,None,click,,22121,77,0.00741,None,None,None,22121,77,0.00741


In [30]:
item_dict.count()

16306

In [31]:
drop_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_ITEM_DICT, hive)

In [32]:
%%time
create_table_from_df(SBX_TEAM_DIGITCAMP, RECSYS_STORY_ITEM_DICT, item_dict, hive)

CPU times: user 12.3 ms, sys: 15.9 ms, total: 28.2 ms
Wall time: 1min 11s


In [3]:
dict_load = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_ITEM_DICT, hive)

In [4]:
dict_load.count()

16306

In [35]:
dict_load.select("product", "item_id", "item_user_count", "item_user_percent").distinct().filter("product <> ''").count()

110

In [10]:
product_list = ['leasing',
                'guarantee',
                'rabota_ru',
                'merchant-acquiring',
                'zarplata',
                'korpkarta',
                'two_gis',
                'evotor',
                'e-acquiring'
                'mybuch',
                'sberrating',
                'credit_card',
                'mybuch_online_ip',
                'lyurist',
                'kredit',
                'bip']

In [11]:
product_list

['leasing',
 'guarantee',
 'rabota_ru',
 'merchant-acquiring',
 'zarplata',
 'korpkarta',
 'two_gis',
 'evotor',
 'e-acquiringmybuch',
 'sberrating',
 'credit_card',
 'mybuch_online_ip',
 'lyurist',
 'kredit',
 'bip']

In [12]:
dict_load.select("product", "item_id", "item_user_count", "item_user_percent")\
          .distinct()\
          .orderBy(f.col("item_user_count").desc())\
          .filter("product <> ''")\
          .filter(f.col("product").isin(product_list))\
          .show(110)

,product,item_id,item_user_count,item_user_percent
0,kredit,214230946,5203,0.50096
1,merchant-acquiring,214243813,4480,0.43135
2,zarplata,214338729,3718,0.35798
3,rabota_ru,214300159,2849,0.27431
4,sberrating,214314599,2606,0.25091
5,lyurist,214240895,2362,0.22742
6,credit_card,214204148,2303,0.22174
7,korpkarta,214228698,1802,0.17350
8,bip,214198011,1433,0.13797
9,evotor,214224178,823,0.07924


In [5]:
dict_load.filter('product like "%bip%"').show()

,hitPagePath,eventCategory,eventAction,product,event_id,event_user_count,event_user_percent,prod_id,prod_user_count,prod_user_percent,item_id,item_user_count,item_user_percent
0,/spasibo-business,[std]: shop,click,bip,105110253,193,0.01858,214198011,1433,0.13797,214198011,1433,0.13797
1,/shop/activated-products,[std]: shop,click,bip,105012110,9875,0.95079,214198011,1433,0.13797,214198011,1433,0.13797
2,/guest-area,[std]: shop,click,bip,104965194,667,0.06422,214198011,1433,0.13797,214198011,1433,0.13797
3,/shop/all-products,[std]: shop,click,bip,105028375,43083,4.14814,214198011,1433,0.13797,214198011,1433,0.13797
4,/main,[operations]: offers,open card,bip,107108535,40477,3.89723,214198011,1433,0.13797,214198011,1433,0.13797
